### Метрики энкодера на основе Resnet18.  
#### Выходной слой: *nn.Linear(in_features=512, out_features=1024, bias=True)*

### Визуализация в 3 ГК помимо того что не дает колличественных оценок точности энкодера, так и несет в себе в лучшем случае около 40% информации от выходного вектора длинной 1024. 

###  Необходимо ознакомится с метриками и оценками модели энкодера. исп.:
* kMeans
* OneClass SVM
* Gaussian Mixture

### Конечная цель: оценка целесообразности применения энкодера в рамках *данной* задачи.

In [ ]:
import albumentations as A
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
from torch import nn
import seaborn as sns
import pandas as pd
import os
import pathlib
import shutil
import cv2
import PIL
import cv2
import sys
from datetime import datetime

TEXT_COLOR = 'black'
# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
%matplotlib inline
plt.rcParams["figure.figsize"] = (17,10)

USE_COLAB_GPU = False
IN_COLAB = False

try:
    import google.colab
    IN_COLAB = True
    USE_COLAB_GPU = True
    from google.colab import drive
except:
    if IN_COLAB:
        print('[!] YOU ARE IN COLAB, BUT DIDNT MOUND A DRIVE. Model wont be synced[!]')

        if not os.path.isfile(CURRENT_FILE_NAME):
            print("FIX ME")
        IN_COLAB = False

    else:
        print('[!] RUNNING NOT IN COLAB')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
if not IN_COLAB:
    PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))
else:
    PROJECT_ROOT = pathlib.Path('..')
    
DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'
SRC_PATH = str(PROJECT_ROOT / 'src')

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

In [ ]:
from torchvision.models import resnet
encoder = resnet.resnet18(pretrained=True)
encoder.fc = nn.Linear(in_features=512, out_features=1024, bias=True)
r = encoder.load_state_dict(torch.load('last_encoder_1024_98')['model'])
encoder.eval()
assert r

In [ ]:
@torch.no_grad()
def simpleGetAllEmbeddings(model, dataset, batch_size, dsc=''):
    
    dataloader = getDataLoaderFromDataset(
        dataset,
        shuffle=True,
        drop_last=False
    )
    
    s, e = 0, 0
    pbar = tqdm(
        enumerate(dataloader), 
        total=len(dataloader),
        position=0,
        leave=False,
        desc='Getting all embeddings...' + dsc)
    info_arr = []
    
    add_info_len = None
    
    for idx, (data, labels, info) in pbar:
        data = data.to(device)
        
        q = model(data)
        
        if labels.dim() == 1:
            labels = labels.unsqueeze(1)
        if idx == 0:
            labels_ret = torch.zeros(
                len(dataloader.dataset),
                labels.size(1),
                device=device,
                dtype=labels.dtype,
            )
            all_q = torch.zeros(
                len(dataloader.dataset),
                q.size(1),
                device=device,
                dtype=q.dtype,
            )
        
        info = np.array(info)
        if add_info_len == None:
            add_info_len = info.shape[0]
        
        info_arr.extend(info.T.reshape((-1, add_info_len)))
        e = s + q.size(0)
        all_q[s:e] = q
        labels_ret[s:e] = labels
        s = e  
    
    all_q = torch.nn.functional.normalize(all_q)
    return all_q, labels_ret, info_arr

### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator, batch_size):
    model.eval()
    train_embeddings, train_labels, _ = simpleGetAllEmbeddings(model, train_set, batch_size, ' for train')
    test_embeddings, test_labels, _ = simpleGetAllEmbeddings(model, test_set, batch_size, ' for test')
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, train_embeddings, test_labels, train_labels, False
    )
    print(accuracies)
    # print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))
    return accuracies["precision_at_1"]


### Этап 1.1. Берем RTDS, из него берем *train* как *baseline*. Заменяем *valid* на *test*.

In [ ]:
RTDS_DF = pd.read_csv(DATA_DIR / 'RTDS_DATASET.csv')
RTDS_DF['filepath'] = RTDS_DF['filepath'].apply(lambda x: str(DATA_DIR / x))
RTDS_DF.drop_duplicates(subset=['filepath'], inplace=True)
RTDS_DF['SET'] = RTDS_DF['SET'].apply(lambda x: 'test' if x == 'valid' else x)

# print(set(RTDS_DF['SET']))
MAKE_ONCE_MERGE_FLAG = True
RTDS_DF

### Этап 1.2. Формируем DataFrame отсутствущих знаков в RTDS.

#### Напоминание чего не хватает. Обр. внимание на 3.25. Там для каждоый скорости потенциально.

| Знак | Описание | Источник |
| ------------- | ------------- | ---- |
| 1.6 | Пересечение равнозначных дорог | - |
| 1.31 | Туннель | - |
| 2.4 | Уступите дорогу | GTSRB Recognition |
| 3.21 | Конец запрещения обгона | GTSRB Recognition |
| 3.22 | Обгон грузовым автомобилям запрещен | GTSRB Recognition |
| 3.23 | Конец запрещения обгона грузовым автомобилям | GTSRB Recognition |
| 3.24-90 | Огр 90 | - |
| 3.24-100 | Огр 100 | GTSRB Recognition |
| 3.24-110 | Огр 110 | - |
| 3.24-120 | Огр 120 | GTSRB Recognition |
| 3.24-130 | Огр 130 | - |
| **3.25** | **Конец огр. максимальной скорости** | **GTSRB Recognition** |
| 3.31 | Конец всех ограничений | GTSRB Recognition |
| 6.3.2 | Зона для разворота | - |

In [ ]:
additional_DF = pd.DataFrame(columns=RTDS_DF.columns)

encode_offset = max(set(RTDS_DF['ENCODED_LABEL'])) + 1
files = os.listdir(DATA_DIR / 'additional_sign')

FLAG_FIG_3_25 = True

sign_list = list(set([x.split('_')[0] for x in files]))
for file in files:
    sign = file.split('_')[0]
    # print(file.split('_')[1].split('.')[0])
    encoded_label = encode_offset + int(sign_list.index(sign))
    
    if FLAG_FIG_3_25 and sign.rsplit('.', 1)[0] == '3.25':
        sign = '3.25'
                
    # print(sign)
    row = {'filepath': str(DATA_DIR / 'additional_sign' / file), 
           'SIGN': sign, 
           'ENCODED_LABEL': encoded_label, 
           'SET': 'test'
          } 
    additional_DF = additional_DF.append(row, ignore_index=True)

if 'index' in additional_DF.columns:
    additional_DF.drop('index', axis=1, inplace=True)
    
display(additional_DF)
if MAKE_ONCE_MERGE_FLAG:
    RTDS_DF = RTDS_DF.append(additional_DF).reset_index()
    MAKE_ONCE_FLAG = True
    
additional_DF.groupby('SIGN').agg(['count'])

In [ ]:
LABEL_DICT = dict(zip(RTDS_DF.SIGN, RTDS_DF.ENCODED_LABEL))
LABEL_DICT

### Этап 2. Формируем для отсутствующих знаков baseline из образцовых знаков с википедии.

In [ ]:
STOCK_SIGNS_CSV_LOCATION = DATA_DIR / 'STOCK_SIGNS.csv'
STOCK_SIGNS_DATAFRAME = pd.read_csv(STOCK_SIGNS_CSV_LOCATION)

STOCK_SIGNS_DATAFRAME.loc[STOCK_SIGNS_DATAFRAME['SIGN'] == '5.19.2', 'SIGN'] = '5.19.1'
if FLAG_FIG_3_25:
    STOCK_SIGNS_DATAFRAME['SIGN'] = STOCK_SIGNS_DATAFRAME['SIGN'].apply(
        lambda x: '3.25' if x.rsplit('.', 1)[0] == '3.25' else x)

## FIXUP для проблем описанных ниже
STOCK_SIGNS_DATAFRAME['SIGN'] = STOCK_SIGNS_DATAFRAME['SIGN'].apply(
        lambda x: '3.18' if x.rsplit('.', 1)[0] == '3.18' else x)

STOCK_SIGNS_DATAFRAME['SIGN'] = STOCK_SIGNS_DATAFRAME['SIGN'].apply(
        lambda x: '2.3' if x.rsplit('.', 1)[0] == '2.3' else x)

STOCK_SIGNS_DATAFRAME['filepath'] = STOCK_SIGNS_DATAFRAME['filepath'].apply(lambda x: str(DATA_DIR / x))
STOCK_SIGNS_DATAFRAME['ENCODED_LABEL'] = [LABEL_DICT[i] for i in STOCK_SIGNS_DATAFRAME['SIGN']]

STOCK_SIGNS_DATAFRAME[::6]

## БЕДА, RTDS объеденяет все 3.18 в одну группу. Еще проблемные: 2.3.1. Ну пофиг блин) Смотрим на ошибки.

### Baseline готов, тестовый датасет готов. Че хотим? Хотим получить какие-нибудь метрики.

In [ ]:
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.augmentations.transforms import PadIfNeeded
from albumentations.augmentations.geometric.resize import LongestMaxSize

img_size = 40

minimal_transform = A.Compose(
        [
        LongestMaxSize(
            img_size,
            # interpolation=cv2.INTER_LANCZOS4 
        ),
        PadIfNeeded(
            img_size, 
            img_size, 
            border_mode=cv2.BORDER_CONSTANT, 
            value=0
        ),
        ToTensorV2(),
        ]
    )

class SignDataset(torch.utils.data.Dataset):
    def __init__(self, df, set_label=None, hyp=None, transform=None, alpha_color=None):
                
        self.transform = transform
        
        if set_label == None:
            self.df = df
        else:
            self.df = df[df['SET']==set_label]
        
        self.hyp = hyp
        self.alpha_color = alpha_color
    def __len__(self):
        return len(self.df.index)
    
    def __getitem__(self, index): 
        label = int(self.df.iloc[index]['ENCODED_LABEL'])
        path = str(self.df.iloc[index]['filepath'])
        sign = str(self.df.iloc[index]['SIGN'])
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        
        # check does it contains transparent channel 
        if img.shape[2] == 4:
        # randomize transparent
            trans_mask = img[:,:,3] == 0
            img[trans_mask] = [self.alpha_color if self.alpha_color else random.randrange(0, 256), 
                               self.alpha_color if self.alpha_color else random.randrange(0, 256), 
                               self.alpha_color if self.alpha_color else random.randrange(0, 256), 
                               255]

            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        # /randomize transparent
                
        # augment 
        if self.transform:
            img = self.transform(image=img)['image']
        # /augment
        
        img = img / 255
        return img, label, (path, sign)

train_dataset = SignDataset(RTDS_DF, 
                            set_label=None, 
                            transform=minimal_transform, 
                            hyp=None)

valid_dataset = SignDataset(STOCK_SIGNS_DATAFRAME, 
                            set_label=None,  
                            transform=minimal_transform, 
                            hyp=None,
                            alpha_color=144
                           )

nrows, ncols = 70, 6
fig = plt.figure(figsize = (16,200))

for idx, (img, encoded_label, (path, sign)) in enumerate(valid_dataset):
    
    img = torch.Tensor.permute(img, [1, 2, 0]).numpy() 
    ax = fig.add_subplot(nrows, ncols, idx+1)
        
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), aspect=1)
    ax.set_title(str(sign), fontsize=15)
    
plt.tight_layout()

In [ ]:
len(valid_dataset)